In [9]:
# import Pkg
# Pkg.add("DataFrames")
# Pkg.add("CSV")
# Pkg.add("IntervalSets")
# Pkg.add("LatexStrings")

In [10]:
using DataFrames, CSV
using IntervalArithmetic
using LaTeXStrings
using Statistics
# using Base
include("./src/interval-weight-estimation.jl")
include("./src/display/display-crisp-matrix.jl")
include("./src/display/display-interval-vector.jl")
include("./src/display/display-vector.jl")
include("./src/crisp-pcm.jl")
include("./src/importance-estimation.jl")
include("./src/solve-interval-ahp.jl")

LoadError: LoadError: At /workspaces/AHP_Terauchi/src/solve-interval-ahp.jl:197: `@constraint(model, Σl = d⃰)`: No constraint expression detected. If you are trying to construct an equality constraint, use `==` instead of `=`.
in expression starting at /workspaces/AHP_Terauchi/src/solve-interval-ahp.jl:197
in expression starting at /workspaces/AHP_Terauchi/src/solve-interval-ahp.jl:142
in expression starting at /workspaces/AHP_Terauchi/src/interval-weight-estimation.jl:3

In [11]:
# 評価基準数
N = 5;

In [12]:
# 真の区間重要度を読み込み
df = CSV.File("./src/data/N5_Given_interval_weight.csv", header=false) |> DataFrame
T = Vector{Interval{Float64}}() # Float64のところにTを使うとおかしくなるのはなぜか？
for i in 1:N
    push!(T, interval(df[1, 2*i-1], df[1, 2*i]))
end

display(L"T = %$(intervalVectorLaTeXString(T))")

UndefVarError: UndefVarError: `intervalVectorLaTeXString` not defined

In [13]:
# 一対比較行列を1000個読み込み
df1 = CSV.File("./src/data/N5_a3_C_PCM_int.csv", header=false) |> DataFrame

# データフレームをN×Nのサブデータフレームに分割
subdfs = split_dataframe(df1, N);

UndefVarError: UndefVarError: `split_dataframe` not defined

In [14]:
# 結果格納用
P = Matrix{Float64}(undef, N, 1000)
Q = Matrix{Float64}(undef, N, 1000)
R = Matrix{Float64}(undef, N, 1000)
F = Matrix{Float64}(undef, N, 1000)

# 1000個のデータで推定
for (i, subdf) in enumerate(subdfs)

    # 一対比較行列を取り出す
    A = Matrix(subdf)
    # display(L"A = %$(matrixLaTeXString(A))")

    # 提案手法で区間重要度推定
    solution = solveIntervalAHP(A, AD)
    E = solution.W # 区間重要度の推定値
    # display(L"E = %$(intervalVectorLaTeXString(E))")
    # display(L"T = %$(intervalVectorLaTeXString(T))")

    TcapE = T .∩ E # 真の区間重要度と推定値の積集合
    TcupE = T .∪ E # 真の区間重要度と推定値の和集合
    # display(L"T ∩ E = %$(intervalVectorLaTeXString(TcapE))")
    # display(L"T ∪ E = %$(intervalVectorLaTeXString(TcupE))")
    
    # P, Q, R, F の計算
    P[:, i] = calculate_P(T, E) # 真の区間重要度と推定値の積集合の割合
    Q[:, i] = calculate_Q(T, E) # 真の区間重要度と推定値の積集合の割合
    R[:, i] = calculate_R(T, E) # 真の区間重要度と推定値の積集合の割合
    F[:, i] = calculate_F(T, E) # F値
    display(L"P = %$(VectorLaTeXString(P[:,i]))")
    display(L"Q = %$(VectorLaTeXString(Q[:,i]))")
    display(L"R = %$(VectorLaTeXString(R[:,i]))")
    display(L"F = %$(VectorLaTeXString(F[:,i]))")
end

UndefVarError: UndefVarError: `subdfs` not defined

In [15]:
P_ave = mean(P)
Q_ave = mean(Q)
R_ave = mean(R)
F_ave = mean(F)
display(L"Pₐᵥₑ = %$(round(P_ave, digits=5))")
display(L"Qₐᵥₑ = %$(round(Q_ave, digits=5))")
display(L"Rₐᵥₑ = %$(round(R_ave, digits=5))")
display(L"Fₐᵥₑ = %$(round(F_ave, digits=5))")

L"$Pₐᵥₑ = -3.8302348425794414e281$"

L"$Qₐᵥₑ = 0.0$"

L"$Rₐᵥₑ = NaN$"

L"$Fₐᵥₑ = 3.1829$"

In [16]:
P_var = var(P)
Q_var = var(Q)
R_var = var(R)
F_var = var(F)
display(L"Pᵥₐᵣ = %$(round(P_var, digits=5))")
display(L"Qᵥₐᵣ = %$(round(Q_var, digits=5))")
display(L"Rᵥₐᵣ = %$(round(R_var, digits=5))")
display(L"Fᵥₐᵣ = %$(round(F_var, digits=5))")

L"$Pᵥₐᵣ = Inf$"

L"$Qᵥₐᵣ = 0.0$"

L"$Rᵥₐᵣ = NaN$"

L"$Fᵥₐᵣ = 5.56641$"